Task 2

In [2]:
from pymongo import MongoClient
from pymongo import ReadPreference

In [3]:
CONNECTION_STRING = "mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=rs0"
COLLECTION = {"name": "Valery Pupkin"}

In [5]:
# Connect to the replica set
client = MongoClient(
    CONNECTION_STRING,
    read_preference=ReadPreference.PRIMARY
)

# Use the primary read preference mode
print("Primary Read Preference Mode:")
print(client.db.test.find_one())


client = MongoClient(
    CONNECTION_STRING,
    read_preference=ReadPreference.SECONDARY
)

# Use the secondary read preference mode
print("\nSecondary Read Preference Mode:")
print(client.db.test.find_one())

Primary Read Preference Mode:
None

Secondary Read Preference Mode:
None


Task 3

In [18]:
print(client.db.test.find_one({}))

None


In [19]:
client = MongoClient(
    CONNECTION_STRING,
    w=3,
    socketTimeoutMS=None
)

result = client.db.test.insert_one(COLLECTION)


In [20]:
print(client.db.test.find_one({}))

{'_id': ObjectId('65fef4a0dcbbdc79f5bfe1bd'), 'name': 'Valery Pupkin'}


As expected there would be infinite loop if the server is down.
And if I turn the machine on it ends and writes.

Task 4

In [21]:
client = MongoClient(
    CONNECTION_STRING,
    w=3,
    socketTimeoutMS=1000
)
result = client.db.test.insert_one(COLLECTION)

NetworkTimeout: localhost:27018: timed out (configured timeouts: socketTimeoutMS: 1000.0ms, connectTimeoutMS: 20000.0ms)

In [25]:
from pymongo.read_concern import ReadConcern

collection = client.db.get_collection("test", read_concern=ReadConcern("majority"))
collection.find_one(COLLECTION)

{'_id': ObjectId('65fef4a0dcbbdc79f5bfe1bd'), 'name': 'Valery Pupkin'}

So it did wrote it.

## Task 5

In [31]:
# first checking that db is empty
client.db.test.delete_many({})
client = MongoClient(
    "mongodb://localhost:27017"
)
print(client.db.test.find_one({}))

None


In [33]:
# now turning it off and writing to replicate set
client = MongoClient(
    CONNECTION_STRING
)
client.db.test.insert_one(COLLECTION)

InsertOneResult(ObjectId('65fef7a7dcbbdc79f5bfe1c4'), acknowledged=True)

In [34]:
# now turning it back on and checking that it has replicated
client = MongoClient(
    "mongodb://localhost:27017"
)
print(client.db.test.find_one({}))

{'_id': ObjectId('65fef7a7dcbbdc79f5bfe1c4'), 'name': 'Valery Pupkin'}


## Task 6

In [14]:
client.db.test.delete_many({})

In [5]:
# Shutting down both slaves and writing with write concern 1
client = MongoClient(
    CONNECTION_STRING,
    w=1
)
client.db.test.insert_one(COLLECTION)
print(client.db.test.find_one({}))


{'_id': ObjectId('65fefd213e43f460d7ef3ceb'), 'name': 'Valery Pupkin'}


In [7]:
from pymongo.read_concern import ReadConcern

client = MongoClient(
    CONNECTION_STRING,
    read_preference=ReadPreference.PRIMARY_PREFERRED
)

try:
    collection = client.db.get_collection("test", read_concern=ReadConcern("majority"))
    collection.find_one(COLLECTION, max_time_ms=5000)
except:
    print("Doesnt work with majority")

try:
    collection = client.db.get_collection("test", read_concern=ReadConcern("local"))
    collection.find_one(COLLECTION, max_time_ms=5000)
except:
    print("Doesnt work with local")

try:
    collection = client.db.get_collection("test", read_concern=ReadConcern("linearizable"))
    collection.find_one(COLLECTION, max_time_ms=5000)
except:
    print("Doesnt work with lineralizable")


Doesnt work with lineralizable


I don't know why it wroks with majority

In [9]:
# Now turning of the master and turning on slaves and then turning on master
client = MongoClient(
     "mongodb://localhost:27017"
)
collection.find_one({}, max_time_ms=5000)

{'_id': ObjectId('65fefd213e43f460d7ef3ceb'), 'name': 'Valery Pupkin'}

This doesnt work that way either

## Task 7

In [10]:
from time import time

In [13]:
client = MongoClient(
    CONNECTION_STRING
)

client.db.test.insert_one(COLLECTION)
was = time()
collection = client.db.get_collection("test", read_concern=ReadConcern("linearizable"))
collection.find_one({})
print(f"It took {time() - was}")

client.db.test.insert_one({"name": "Vasya Bubkin"})
was = time()
collection = client.db.get_collection("test", read_concern=ReadConcern("linearizable"))
collection.find_one({})
print(f"It took {time() - was}")

ExecutionTimeout: operation exceeded time limit, full error: {'ok': 0.0, 'errmsg': 'operation exceeded time limit', 'code': 50, 'codeName': 'MaxTimeMSExpired', '$clusterTime': {'clusterTime': Timestamp(1711211113, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1711211113, 1)}